In [ ]:
import pandas as pd

In [ ]:
start = "2022-01-01"
end = "2026-01-29"
ticker = 'QQQM'

- Momentum (RSI) shows general strength [Lead indicator]

#### Trend indicators (SMA, MACD) helps to identify direction of the trend. [Lag indicator]
##### SMA8 > SMA20 >> Up trend.
##### Need to apply same time frame as leading indicator (1 hour, daily etc.)
##### Eg. If RSA > 50 and SMA8 > SMA20 in the same (hourly/ daily) candle, buy. But 90% this will not happen and we won't make money,

### Index Long side strategy
#### Time Frame: Daily
#### RSI(7) - 7 periods
#### SMA(8)
#### SMA(20)
#### MACD(12,26,9)
#### If RSI(7) > 50 >> Buy [Momentum]
#### Wait for x candles for either trend conditions to become true
#### 1. SMA(8) > SMA(20) -> Uptrenddata['Long_Entry'] = macd_window.rolling(lookahead_days, min_periods=1).max().astype(bool)

#### 2. MACD(12,26,9) -> Uptrend
##### Exit condition 1:
If SMA or MACD cross up happens within x day, we will hold on to our position. 
If SMA cross up or MACD do not happen within x day, we exit.
##### Exit Condition 2:
If SMA cross up or MACD happens within x days, 
then hold position til either SMA cross down or MACD cross down (MACD histogram > 0)
- 4-5 years historical data

In [ ]:
from data.data_utils import dl_ticker_hist
hist_data = dl_ticker_hist(ticker, start, end)

In [ ]:
FAST_SMA_PERIOD = 8
SLOW_SMA_PERIOD = 20
FAST_MACD_PERIOD = 12
SLOW_MACD_PERIOD = 26
MACD_SIGNAL_PERIOD = 9
RSI_PERIOD = 14

from src.backtester.strategies import sma_macd_rsi
df = sma_macd_rsi(hist_data, sma_fast_period=FAST_SMA_PERIOD, sma_slow_period=SLOW_SMA_PERIOD, macd_fast_period=FAST_MACD_PERIOD, macd_slow_period=SLOW_MACD_PERIOD, macd_signal_period=MACD_SIGNAL_PERIOD, rsi_period=RSI_PERIOD)

In [ ]:
df.info()
df.dtypes

In [ ]:


from src.backtester.plot import plot_sma
plot_sma(df, ticker, FAST_SMA_PERIOD, SLOW_SMA_PERIOD)

In [ ]:

macd_df = df[['macd','macd_signal','macd_histogram']]
from src.backtester.plot import plot_macd
plot_macd(macd_df, ticker)

- When MACD line > signal line, histogram is positive
    - Shows bullish momentum and indicates potential long signal

In [ ]:
print(df.dtypes)

In [ ]:
from src.backtester.plot import plot_rsi
plot_rsi(df, ticker, RSI_PERIOD)

In [ ]:
display(df)

### Exit Condition 2:
#### If SMA cross up or MACD happens within x days, hold position til either SMA cross down or MACD cross down (MACD histogram > 0)

## Backtest Strategy
#### Market-On-Close Execution
- Execute trade at or as close as possible to the official closing price of a trading day
- Signal at today's close -> trade executed at today's close -> PnL starts tomorrow


In [ ]:
# Backtest strategy
initial_capital = 100000
allocation = 0.5 #50% of capital
cost_per_shr = 0.005 # USD per share. Sometimes its cost_per_trade


In [ ]:
display(df[(df['long_entry']==True) ]) #| (df['long_exit']==True)
# display(df[(df['position']==1)])

In [ ]:
from src.backtester.portfolio import generate_portfolio
df = generate_portfolio(df, allocation, initial_capital, cost_per_shr)
display(df[['date', 'long_entry', 'long_exit', 'holding', 'position', 'position_shrs']])
display(df[df['trade']!=''][['date', 'long_entry', 'long_exit', 'holding', 'position', 'position_shrs','trade']])

In [ ]:
import pytz
from datetime import datetime as dtt

date_str = dtt.now(pytz.timezone('America/New_York')).strftime('%Y_%m_%d')
strategy_id = f'sma_macd_rsi_{date_str}'


In [ ]:
# Metrics
from src.metrics.performance import calc_metrics

metrics_df = calc_metrics(df, ticker, strategy_id)

In [ ]:
from src.metrics.trade import calc_trades, calc_trades_metrics, get_recent_trades
import os
metrics_dir = f'metrics/{strategy_id}'
if metrics_dir is not None:
    os.makedirs(metrics_dir, exist_ok=True)
trades_df = calc_trades(df)
trade_metrics_df = calc_trades_metrics(df, ticker, strategy_id, metrics_dir)
recent_trades_df = get_recent_trades(trades_df, ticker, strategy_id, metrics_dir)


In [ ]:
import os
metrics_dir = f'results/metrics/{strategy_id}'
os.makedirs(metrics_dir, exist_ok=True)
display(trade_metrics_df)


In [ ]:

metrics_df = pd.concat([metrics_df, trade_metrics_df], axis=1)
print(f'\nmetrics_df: \n{metrics_df}')
metrics_df.to_csv(f'{metrics_dir}/metrics_{strategy_id}_{ticker.lower().replace("-","_")}.csv', index=False)



In [ ]:
from src.metrics.performance import get_all_metrics_by_strategy
all_metrics_df = get_all_metrics_by_strategy(strategy_id)
display(all_metrics_df)

#### Sharpe ratio 1+ decent, 1.5 very good. 2+ excellent
#### Max drawdown 20-40%
#### Win rate 40-60%
#### Avg pnl per trade > 0



## Exit Condition 2:
### If SMA cross up or MACD happens within x days, hold position til either SMA cross down or MACD cross down (MACD histogram > 0)

In [ ]:
exit_df = df.copy()
exit_df = exit_df[exit_df['long_exit']==True]
display(exit_df[['date', 'ticker', 'close_px', 'rsi_long_signal', 'sma_cross_long', 'macd_cross_long', 'long_exit']])

## Backtest Strategy
#### Market-On-Close Execution
- Execute trade at or as close as possible to the official closing price of a trading day
- Signal at today's close -> trade executed at today's close -> PnL starts tomorrow


In [ ]:
# import numpy as np
# # Backtest strategy
# initial_capital = 100000
# allocation = 0.5 #50% of capital


# df['position'] = 0
# # df.loc[df['long_entry'],'position'] = 1
# # Only enter if long_entry is True AND position is flat yesterday
# df.loc[(df['long_entry'] & (~df['long_entry'].shift(1).fillna(False))), 'position'] = 1
# # display(df[df['date'].isin(['2022-02-09', '2022-02-10', '2022-02-11'])][['date', 'long_entry', 'long_exit', 'position']])

# # Set to 0 if today is long_exit and yesterday's position > 0
# df.loc[df['long_exit'] & (df['position'].shift(1) > 0), 'position'] = 0
# df['position'] = df['position'].ffill().fillna(0)
# display(df[df['date'].isin(['2022-02-07', '2022-02-08', '2022-02-09'])][['date', 'long_entry', 'long_exit', 'position']])

# # df['position'] = df['position'].astype(float)
# # df['position'] = df['position'].replace(0, np.nan).ffill().fillna(0)

# #scale position by allocation (dollar-based position)
# df['position'] = df['position'] * (allocation * initial_capital)
# df['position_shrs'] = 0
# df.loc[df['long_entry'],'position_shrs'] = df['position'] // df['close_px'] # Can use / for fractional shares
# df['position_shrs'] = df['position_shrs'].replace(0, np.nan).ffill().fillna(0)
# df.loc[df['long_exit'],'position_shrs'] = 0

# display(df[['date', 'long_entry', 'long_exit', 'position', 'position_shrs']])
# # display(df[df['position']>0])



In [ ]:
#Transaction costs
cost_per_shr = 0.005 # USD per share. Sometimes its cost_per_trade
# df['position'] = df['position'].astype(int)
# df['trade'] = df['position'].diff().abs()
df['trade_shrs'] = 0
df.loc[df['trade']=='BUY', 'trade_shrs'] = df['position_shrs']
df['trade_cost'] = df['trade_shrs'] * cost_per_shr

# df.loc[df['long_entry']==True,'trade'] = df['position']
display(df[df['trade']!=''])
display(df)

In [ ]:
# Metrics
total_return = round((df['cum_ret'].iloc[-1] - 1)*100, 2)
print(f'total_return: {total_return}%')

sharpe = round((df['daily_ret_c2c'].mean() / df['daily_ret_c2c'].std() * np.sqrt(252)),2)
print(f'sharpe: {sharpe}%')

max_drawdown = round(((df['cum_ret'] / df['cum_ret'].cummax() - 1).min()), 2)
print(f'max_drawdown: {max_drawdown}%')

total_pnl = round(df['daily_pnl'].sum(), 2)
print(f'total_pnl: ${total_pnl}')

metrics_df = pd.DataFrame({
    'ticker': [ticker],
    'total_return': [total_return],
    'sharpe_ratio': [sharpe],
    'max_drawdown': [max_drawdown],
    'total_pnl': [total_pnl],
})
# print(f'\nmetrics_df: \n{metrics_df}')
# metrics_df.to_csv(f'metrics_{ticker.lower().replace("-","_")}', index=False)

In [ ]:

display(trades_df)
recent_trades.to_csv(f'metrics/recent_trades_{ticker.lower().replace("-","_")}.csv', index=False)
trades_df.to_csv(f'metrics/trades_{ticker.lower().replace("-","_")}.csv', index=False)

In [ ]:
# Cumulative PnL / Equity Curve

import matplotlib.ticker as mtick
plt.figure(figsize=(12,6))
plt.plot(df['date'], df['cum_pnl'], label='Equity Curve', color='blue')
plt.axhline(initial_capital, color='orange', linestyle='--', label='Initial Capital')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.title(f'{ticker} Strategy Equity Curve')
plt.grid(True)
plt.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
plt.legend()
plt.show()
plt.savefig(f'{ticker.lower().replace("-","_")}_equity_curve.png', dpi=300)


In [ ]:
# aiq.to_csv('metrics/metrics_aiq.csv', index=False)

#Sharpe ratio 1+ decent, 1.5 very good. 2+ excellent
#Max drawdown 20-40%
#Win rate 40-60%
#Avg pnl per trade > 0
#
